In [ ]:
# Landscape Classification Pipeline Notebook
# ===============================
# 
# Este notebook serve como ponto de entrada para o pipeline completo de classificação de imagens de paisagens naturais.
# O pipeline utiliza várias arquiteturas de deep learning (ResNet50, EfficientNet, Vision Transformer, MobileNet)
# e integra diversos componentes como otimização de hiperparâmetros, ensemble de modelos e visualizações.

import os
import sys
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import yaml
from datetime import datetime
from pathlib import Path
import importlib
import warnings
warnings.filterwarnings('ignore')

# Configurar exibição de gráficos no notebook
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

# Verificar disponibilidade de GPU
print(f"PyTorch versão: {torch.__version__}")
print(f"CUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Dispositivo atual: {device}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memória GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    device = torch.device("cpu")
    print(f"Dispositivo atual: {device}")

# %% [markdown]
# ## 1. Configuração do ambiente

# %%
# Verificar e criar diretórios necessários
def ensure_directories():
    """Verifica e cria os diretórios necessários para o pipeline."""
    directories = [
        "data",
        "models",
        "logs",
        "results",
        "tensorboard_logs",
        "config"
    ]
    for directory in directories:
        os.makedirs(directory, exist_ok=True)
        print(f"✓ Diretório '{directory}' verificado/criado")

ensure_directories()

# %%
# Verificar se o arquivo de configuração existe, caso contrário criar um padrão
CONFIG_PATH = 'config/config.yaml'

if not os.path.exists(CONFIG_PATH):
    print("Arquivo de configuração não encontrado. Criando configuração padrão...")
    
    default_config = {
        'dataset': {
            'path': 'C:\\Dataset\\intel-image-classification',
            'img_size': 224,
            'batch_size': 32
        },
        'training': {
            'epochs': 15,
            'learning_rate': 0.001,
            'gamma': 0.1,
            'step_size': 7,
            'seed': 42,
            'k_folds': 5
        },
        'models': {
            'use_resnet': True,
            'use_efficientnet': True,
            'use_mobilenet': True,
            'use_vit': True
        },
        'flow_control': {
            'run_data_exploration': True,
            'run_dataset_prep': True,
            'run_cross_validation': False,
            'run_optimization': False,
            'run_training': True,
            'run_evaluation': True,
            'run_visualization': True,
            'run_compression': False,
            'run_ensemble': True,
            'run_export': False,
            'run_final_report': True,
            'run_duplicate_detection': True
        }
    }
    
    os.makedirs(os.path.dirname(CONFIG_PATH), exist_ok=True)
    with open(CONFIG_PATH, 'w') as f:
        yaml.dump(default_config, f, default_flow_style=False)
    
    print(f"Configuração padrão criada em {CONFIG_PATH}")
else:
    print(f"Arquivo de configuração encontrado em {CONFIG_PATH}")

# Carregar configuração
with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

print("\nConfiguração atual:")
print(json.dumps(config, indent=2))

# %% [markdown]
# ## 2. Verificação de dependências

# %%
# Verificar a presença dos módulos auxiliares necessários
required_modules = [
    'utils',
    'data_processing',
    'dataset',
    'dataset_utils',
    'models',
    'training',
    'visualization',
    'optimization',
    'model_compression',
    'ensemble',
    'landscape_enhancements'
]

missing_modules = []
for module_name in required_modules:
    try:
        module = importlib.import_module(module_name)
        print(f"✓ Módulo '{module_name}' carregado com sucesso")
    except ImportError:
        missing_modules.append(module_name)
        print(f"✗ Módulo '{module_name}' não encontrado")

if missing_modules:
    print("\nAtenção: Os seguintes módulos estão faltando:")
    for module in missing_modules:
        print(f"  - {module}.py")
    print("\nCertifique-se de que todos os arquivos .py auxiliares estão no mesmo diretório que este notebook.")
else:
    print("\nTodos os módulos necessários estão disponíveis.")

# %% [markdown]
# ## 3. Preparação do Dataset

# %%
# Verificar a presença dos dados ou fornecer instruções para download
dataset_path = config['dataset']['path']

if not os.path.exists(dataset_path):
    print(f"Dataset não encontrado em '{dataset_path}'")
    print("\nInstruções para download:")
    print("1. Faça o download do dataset 'Intel Image Classification' do Kaggle:")
    print("   https://www.kaggle.com/datasets/puneet6060/intel-image-classification")
    print("2. Extraia o arquivo baixado para o diretório 'data/intel-image-classification'")
    print("3. Execute esta célula novamente para verificar")
else:
    # Verificar a estrutura do dataset
    train_dir = os.path.join(dataset_path, 'seg_train', 'seg_train')
    test_dir = os.path.join(dataset_path, 'seg_test', 'seg_test')
    
    # Verificar caminhos alternativos se os padrões não forem encontrados
    if not os.path.exists(train_dir):
        train_dir = os.path.join(dataset_path, 'train')
    
    if not os.path.exists(test_dir):
        test_dir = os.path.join(dataset_path, 'test')
    
    train_exists = os.path.exists(train_dir)
    test_exists = os.path.exists(test_dir)
    
    if train_exists and test_exists:
        print(f"✓ Dataset encontrado e estrutura validada")
        print(f"  - Diretório de treinamento: {train_dir}")
        print(f"  - Diretório de teste: {test_dir}")
        
        # Mostrar um preview do dataset
        import data_processing
        
        # Criar DataFrames com informações sobre as imagens
        df_train, train_corrupted = data_processing.create_df(train_dir)
        df_test, test_corrupted = data_processing.create_df(test_dir)
        
        print(f"\nEstatísticas do Dataset:")
        print(f"  - Total de imagens de treinamento: {len(df_train)}")
        print(f"  - Total de imagens de teste: {len(df_test)}")
        
        if train_corrupted:
            print(f"  - Imagens corrompidas no treinamento: {len(train_corrupted)}")
        if test_corrupted:
            print(f"  - Imagens corrompidas no teste: {len(test_corrupted)}")
        
        # Exibir a distribuição de classes
        train_class_dist = df_train['label'].value_counts()
        print("\nDistribuição de classes (Treinamento):")
        print(train_class_dist)
        
        # Visualizar a distribuição
        plt.figure(figsize=(10, 6))
        ax = sns.barplot(x=train_class_dist.index, y=train_class_dist.values)
        plt.title("Distribuição de Classes - Conjunto de Treinamento")
        plt.ylabel("Número de imagens")
        plt.xlabel("Classe")
        
        # Adicionar valores nas barras
        for i, v in enumerate(train_class_dist.values):
            ax.text(i, v + 10, str(v), ha='center')
        
        plt.tight_layout()
        plt.show()
        
        # Exibir algumas imagens de exemplo
        data_processing.show_random_images(df_train, num_images=9, save_path=None)
    else:
        print(f"✗ Estrutura do dataset inválida")
        if not train_exists:
            print(f"  - Diretório de treinamento não encontrado: {train_dir}")
        if not test_exists:
            print(f"  - Diretório de teste não encontrado: {test_dir}")

# %% [markdown]
# ## 4. Executar o Pipeline Completo

# %%
# Atualizar configurações antes de executar o pipeline
# Você pode modificar os parâmetros de configuração aqui conforme necessário

# Exemplo de atualização de configuração
config['training']['epochs'] = 10  # Reduzir o número de épocas para testes rápidos
config['dataset']['batch_size'] = 16  # Reduzir batch size para economizar memória

# Atualizar flags de controle do fluxo
config['flow_control']['run_cross_validation'] = False  # Desabilitar validação cruzada
config['flow_control']['run_optimization'] = False  # Desabilitar otimização de hiperparâmetros
config['flow_control']['run_compression'] = False  # Desabilitar compressão de modelos

# Adicionar nova flag para detecção de duplicatas
if 'run_duplicate_detection' not in config['flow_control']:
    config['flow_control']['run_duplicate_detection'] = True  # Habilitar detecção de duplicatas

# Para executar apenas a detecção de duplicatas, desabilite as outras etapas
# Descomente estas linhas se quiser executar apenas a detecção de duplicatas
# config['flow_control']['run_dataset_prep'] = False
# config['flow_control']['run_training'] = False
# config['flow_control']['run_evaluation'] = False
# config['flow_control']['run_visualization'] = False
# config['flow_control']['run_ensemble'] = False
# config['flow_control']['run_final_report'] = False

# Salvar a configuração atualizada
with open(CONFIG_PATH, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("Configuração atualizada e salva.")

# %%
# Configurar variáveis de ambiente e importar o main.py

# Importar o módulo main contendo a função principal
import main

# Executar o pipeline principal
print("Iniciando o pipeline de classificação de paisagens...\n")
print("="*80)

try:
    # Executar o pipeline completo
    main.main()
    print("\n✓ Pipeline executado com sucesso!")
except Exception as e:
    print(f"\n✗ Erro durante a execução do pipeline: {str(e)}")
    import traceback
    traceback.print_exc()

# %% [markdown]
# ## 4.1 Visualização de Resultados da Detecção de Duplicatas

# %%
# Verificar e exibir resultados da detecção de duplicatas
duplicates_report_path = "results/duplicate_images.csv"
duplicates_viz_path = "results/duplicates"

if os.path.exists(duplicates_report_path):
    # Carregar o relatório de duplicatas
    duplicates_df = pd.read_csv(duplicates_report_path)
    
    print(f"Resultados da Detecção de Duplicatas:")
    print(f"- Total de imagens duplicadas detectadas: {len(duplicates_df)}")
    print(f"- Número de grupos de duplicatas: {duplicates_df['hash'].nunique()}")
    
    # Mostrar a distribuição de tamanhos dos grupos
    group_sizes = duplicates_df.groupby('hash').size().value_counts().sort_index()
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(group_sizes.index, group_sizes.values)
    plt.title("Distribuição de Tamanhos de Grupos de Imagens Similares")
    plt.xlabel("Tamanho do grupo")
    plt.ylabel("Número de grupos")
    plt.grid(axis='y', alpha=0.3)
    
    # Adicionar rótulos nas barras
    for bar, size in zip(bars, group_sizes.values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, str(size), ha='center')
    
    plt.tight_layout()
    plt.show()
    
    # Exibir exemplos de imagens duplicadas se disponíveis
    duplicate_images = [f for f in os.listdir(duplicates_viz_path) if f.startswith('duplicate_group_') and f.endswith('.png')] if os.path.exists(duplicates_viz_path) else []
    
    if duplicate_images:
        from IPython.display import Image, display
        
        print("\nExemplos de grupos de imagens duplicadas/similares:")
        for img_path in sorted(duplicate_images)[:3]:  # Mostrar até 3 exemplos
            display(Image(filename=os.path.join(duplicates_viz_path, img_path)))
    
else:
    print("Relatório de detecção de duplicatas não encontrado. Execute o pipeline com a opção de detecção de duplicatas habilitada.")

# %% [markdown]
# ## 5. Análise de Resultados

# %%
# Carregar e visualizar os resultados após a execução
results_path = "results/evaluation_results.json"

if os.path.exists(results_path):
    with open(results_path, 'r') as f:
        evaluation_results = json.load(f)
    
    print("Resumo dos resultados:")
    model_accuracies = {}
    for model_name, results in evaluation_results.items():
        accuracy = results['accuracy']
        model_accuracies[model_name] = accuracy
        print(f"{model_name}: Acurácia = {accuracy:.4f}")
    
    # Visualizar comparação de acurácias
    plt.figure(figsize=(10, 6))
    models = list(model_accuracies.keys())
    accuracies = [model_accuracies[name] for name in models]
    
    bars = plt.bar(models, accuracies, color='skyblue')
    plt.title('Comparação de Acurácia entre Modelos', fontsize=14)
    plt.ylabel('Acurácia', fontsize=12)
    plt.ylim(0.5, 1.0)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Adicionar valores nas barras
    for bar, acc in zip(bars, accuracies):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                 f'{height:.4f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # Verificar se resultados do ensemble estão disponíveis
    ensemble_path = "results/ensemble_comparison.png"
    if os.path.exists(ensemble_path):
        from IPython.display import Image
        print("\nResultados do Ensemble:")
        display(Image(filename=ensemble_path))
    
    # Verificar se há visualizações Grad-CAM geradas
    gradcam_dirs = [d for d in os.listdir('results') if d.startswith('gradcam_')]
    if gradcam_dirs:
        print("\nVisualizações Grad-CAM disponíveis para os seguintes modelos:")
        for d in gradcam_dirs:
            print(f"- {d.replace('gradcam_', '')}")
        
        # Mostrar exemplo para o primeiro modelo
        sample_gradcam = os.path.join('results', gradcam_dirs[0], f'gradcam_summary_{gradcam_dirs[0].replace("gradcam_", "")}.png')
        if os.path.exists(sample_gradcam):
            print("\nExemplo de visualização Grad-CAM:")
            display(Image(filename=sample_gradcam))
else:
    print("Resultados de avaliação não encontrados. Execute o pipeline primeiro.")

# %% [markdown]
# ## 6. Funções Auxiliares para Análise Adicional

# %%
def load_model(model_name):
    """
    Carrega um modelo treinado com base no nome.
    
    Args:
        model_name (str): Nome do modelo a ser carregado ('resnet50', 'efficientnet', 'vit', 'mobilenet')
    
    Returns:
        model: Modelo PyTorch carregado
    """
    import models
    import torch
    
    # Carregar hiperparâmetros salvos
    try:
        with open('results/best_hyperparameters.json', 'r') as f:
            params_data = json.load(f)
        
        model_params = params_data.get(model_name, {}).get('params', {})
        
        if model_name == 'resnet50':
            model = models.create_model_with_best_params(model_params, model_name='resnet50')
            model.load_state_dict(torch.load(f'models/{model_name}_final.pth'))
        elif model_name == 'efficientnet':
            model = models.create_model_with_best_params(model_params, model_name='efficientnet')
            model.load_state_dict(torch.load(f'models/{model_name}_final.pth'))
        elif model_name == 'vit':
            vit_params = {
                'model_name': model_params.get('model_type', 'vit_base_patch16_224'),
                'pretrained': True,
                'dropout_rate': model_params.get('dropout_rate', 0.1)
            }
            model = models.create_vit_model(vit_params)
            model.load_state_dict(torch.load(f'models/{model_name}_final.pth'))
        elif model_name == 'mobilenet':
            # Determinar o número de classes a partir do conjunto de dados
            import data_processing
            train_dir = os.path.join(config['dataset']['path'], 'seg_train', 'seg_train')
            if not os.path.exists(train_dir):
                train_dir = os.path.join(config['dataset']['path'], 'train')
            df_train, _ = data_processing.create_df(train_dir)
            num_classes = len(df_train['label'].unique())
            
            model = models.mobilenet_v3_small(weights=None, num_classes=num_classes)
            model.load_state_dict(torch.load(f'models/{model_name}_final.pth'))
        else:
            raise ValueError(f"Modelo não reconhecido: {model_name}")
        
        model.eval()
        return model
    except Exception as e:
        print(f"Erro ao carregar o modelo {model_name}: {str(e)}")
        return None

def predict_single_image(image_path, model_name):
    """
    Faz a predição para uma única imagem.
    
    Args:
        image_path (str): Caminho para a imagem
        model_name (str): Nome do modelo a ser usado ('resnet50', 'efficientnet', 'vit', 'mobilenet')
    
    Returns:
        tuple: (classe_predita, probabilidades)
    """
    import torch
    from PIL import Image
    import torchvision.transforms as transforms
    
    # Carregar o modelo
    model = load_model(model_name)
    if model is None:
        return None, None
    
    # Determinar as classes
    import data_processing
    train_dir = os.path.join(config['dataset']['path'], 'seg_train', 'seg_train')
    if not os.path.exists(train_dir):
        train_dir = os.path.join(config['dataset']['path'], 'train')
    df_train, _ = data_processing.create_df(train_dir)
    classes = sorted(df_train['label'].unique())
    
    # Preparar transformações
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Carregar e transformar a imagem
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0)
    
    # Fazer a predição
    with torch.no_grad():
        outputs = model(img_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)[0].cpu().numpy()
        _, predicted = torch.max(outputs, 1)
        predicted_class = classes[predicted.item()]
    
    return predicted_class, probabilities

# Função para visualizar predições
def visualize_prediction(image_path, model_names=None):
    """
    Visualiza a predição de vários modelos para uma única imagem.
    
    Args:
        image_path (str): Caminho para a imagem
        model_names (list): Lista de nomes de modelos. Se None, usa todos os modelos disponíveis.
    """
    from PIL import Image
    import matplotlib.pyplot as plt
    
    if model_names is None:
        model_names = ['resnet50', 'efficientnet', 'vit', 'mobilenet']
    
    # Carregar a imagem
    img = Image.open(image_path).convert('RGB')
    
    # Determinar as classes
    import data_processing
    train_dir = os.path.join(config['dataset']['path'], 'seg_train', 'seg_train')
    if not os.path.exists(train_dir):
        train_dir = os.path.join(config['dataset']['path'], 'train')
    df_train, _ = data_processing.create_df(train_dir)
    classes = sorted(df_train['label'].unique())
    
    # Configurar o gráfico
    n_models = len(model_names)
    fig, axs = plt.subplots(1, n_models + 1, figsize=(5 * (n_models + 1), 5))
    
    # Mostrar a imagem original
    axs[0].imshow(img)
    axs[0].set_title("Imagem Original")
    axs[0].axis('off')
    
    # Para cada modelo, mostrar as probabilidades de classe
    for i, model_name in enumerate(model_names):
        predicted_class, probabilities = predict_single_image(image_path, model_name)
        
        if predicted_class is not None and probabilities is not None:
            # Mostrar gráfico de barras das probabilidades
            bars = axs[i+1].bar(classes, probabilities)
            axs[i+1].set_title(f"{model_name}\nPredição: {predicted_class}")
            axs[i+1].set_ylim(0, 1)
            axs[i+1].tick_params(axis='x', rotation=45)
            
            # Destacar a classe predita
            idx = classes.index(predicted_class)
            bars[idx].set_color('red')
    
    plt.tight_layout()
    plt.show()

# %%
# Exemplo de uso das funções auxiliares
# Para usar, descomente e substitua pelo caminho real da imagem
# image_path = "caminho/para/sua/imagem.jpg"
# visualize_prediction(image_path)

# %% [markdown]
# ## 7. Conclusão

# %%
# Carregar e exibir o resumo do experimento 
summary_files = [f for f in os.listdir('results') if f.startswith('experiment_summary_') and f.endswith('.txt')]

if summary_files:
    # Ordenar por data (assumindo o formato YYYYMMDD_HHMMSS no nome do arquivo)
    latest_summary = sorted(summary_files)[-1]
    
    print(f"Resumo do experimento mais recente ({latest_summary}):")
    print("="*80)
    
    with open(os.path.join('results', latest_summary), 'r') as f:
        print(f.read())
    
    # Exibir visualização de resumo se disponível
    summary_image = latest_summary.replace('.txt', '.png')
    if os.path.exists(os.path.join('results', summary_image)):
        from IPython.display import Image
        print("\nVisualização do resumo:")
        display(Image(filename=os.path.join('results', summary_image)))
else:
    print("Não foram encontrados resumos de experimentos. Execute o pipeline completo primeiro.")

# %%
# Resumo do notebook e próximos passos
print("Resumo do Pipeline de Classificação de Paisagens Naturais")
print("="*60)
print("\nEste notebook integrou o pipeline completo que inclui:")
print("1. Preparação e exploração do dataset")
print("2. Treinamento de múltiplos modelos (ResNet50, EfficientNet, Vision Transformer, MobileNet)")
print("3. Avaliação e visualização de resultados")
print("4. Técnicas avançadas como ensemble de modelos e interpretabilidade visual")
print("\nPróximos passos possíveis:")
print("- Experimente com diferentes hiperparâmetros")
print("- Adicione novas arquiteturas de modelos")
print("- Aplique técnicas de data augmentation mais avançadas")
print("- Implemente quantização e pruning para modelos mais leves")
print("- Aplique o modelo a novas imagens de paisagens")